In [1]:
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, Model, Sequential
import json
import os
import time
import glob
import random
from PIL import Image
import numpy as np
import datetime

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

gpus = tf.config.experimental.list_physical_devices("GPU")
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)
        exit(-1)

In [4]:
log_dir = "./logs/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
train_writer = tf.summary.create_file_writer(os.path.join())
val_writer = tf.summary.create_file_writer(os.path.join(log_dir, "val"))


#data_root = os.path.abspath(os.path.join(os.getcwd(), "../.."))
data_root = './'
image_path = os.path.join(data_root, "flower_data")
train_dir = os.path.join(image_path, "train")
validation_dir = os.path.join(image_path, "val")

In [5]:
# class dict
data_class = [cla for cla in os.listdir(train_dir) if os.path.isdir(os.path.join(train_dir, cla))]
class_num = len(data_class)
class_dict = dict((value, index) for index, value in enumerate(data_class))

# reverse value and key of dict
inverse_dict = dict((val, key) for key, val in class_dict.items())
# write dict into json file
json_str = json.dumps(inverse_dict, indent=4)
with open('class_indices.json', 'w') as json_file:
    json_file.write(json_str)
    
# create direction for saving weights
if not os.path.exists("save_weights"):
    os.makedirs("save_weights")


In [6]:
im_height = 224
im_width = 224

_R_MEAN = 123.68
_G_MEAN = 116.78
_B_MEAN = 103.94

batch_size = 32
epochs = 30

In [8]:
random.seed(0)

train_image_path = glob.glob(train_dir + "/*/*.jpg")
random.shuffle(train_image_path)
train_num = len(train_image_path)
train_label_list = [class_dict[path.split(os.path.sep)[-2]] for path in train_image_path]

val_image_path = glob.glob(validation_dir + "/*/*.jpg")
random.shuffle(val_image_path)
val_num = len(val_image_path)
val_label_list = [class_dict[path.split(os.path.sep)[-2]] for path in val_image_path]

print("using {} images for training, {} images for validation.".format(train_num, val_num))

KeyboardInterrupt: 

In [ ]:
def process_train_img(img_path, label):
    label = tf.one_hot(label, depth=class_num)
    image = tf.io.read_file(img_path)
    image = tf.image.decode_jpeg(image)
    image = tf.cast(image, tf.float32)
    image = tf.image.resize(image, [im_height, im_width])
    image = tf.image.random_flip_left_right(image)
    image = image - [_R_MEAN, _G_MEAN, _B_MEAN]
    return image, label

def process_val_img(img_path, label):
    label = tf.one_hot(label, depth=class_num)
    image = tf.io.read_file(img_path)
    image = tf.image.decode_jpeg(image)
    image = tf.cast(image, tf.float32)
    image = tf.image.resize(image, [im_height, im_width])
    image = image - [_R_MEAN, _G_MEAN, _B_MEAN]
    return image, label

AUTOTUNE = tf.data.experimental.AUTOTUNE

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((tf.constant(train_image_path),
                                                    tf.constant(train_label_list)))
total_train = len(train_image_path)

train_dataset = train_dataset.map(process_train_img, num_parallel_calls=AUTOTUNE)\
                             .shuffle(buffer_size= total_train)\
                             .batch(batch_size)\
                             .prefetch(buffer_size=AUTOTUNE)

val_dataset = tf.data.Dataset.from_tensor_slices((tf.constant(val_image_path),
                                                  tf.constant(val_label_list)))
total_val = len(val_image_path)
val_dataset = val_dataset.map(process_train_img, num_parallel_calls=AUTOTUNE)\
                         .batch(batch_size)\
                         .prefetch(buffer_size=AUTOTUNE)

In [ ]:
class BasicBlock(layers.Layer):
    expansion = 1
    
    def __init__(self, out_channel, stride = 1, downsample=None, **kwargs):
        super(BasicBlock, self).__init__(**kwargs)
        
        self.conv1 = layers.Conv2D(out_channel, kernel_size=3, strides=strides,
                                   padding="SAME", use_bias=False)
        self.bn1 = layers.BatchNormalization(momentum=0.9, epsilon=le-5)
        
        self.conv2 = layers.Conv2D(out_channel, kernel_size=3, strides=1,
                                    padding="SAME", use_bias=False)
        self.bn2 = layers.BatchNormalization(momentum=0.9, epsilon=1e-5)
        
        self.downsample = downsample
        self.relu = layers.ReLU()
        self.add = layers.Add()
        
    def call(self, inputs, training = False):
        identity = inputs
        if self.downsample is not None:
            identity = self.downsample(inputs)
            
        x = self.conv1(inputs)
        x = self.bn1(x, training=training)
        x = self.relu(x)
        
        x = self.conv2(x)
        x = self.bn2(x, training=tfraining)
        
        x = self.add([identity, x])
        x = self.relu(x)
        
        return x
    
    
class Bottleneck(layers.Layer):
    expansion = 4
    
    def __init__(self, out_channel, strides= 1, downsample=None, **kwargs):
        super(Bottleneck, self).__init__(**kwargs)
        self.conv1 = layers.Conv2D(out_channel, kernel_size=1, use_bias=False, name = "conv1")
        self.bn1 = layers.BatchNormalization(momentum=0.9, epsilon=1e-5, name="conv1/BatchNorm")
        
        self.conv2 = layers.Conv2D(out_channel, kernel_size=3, use_bias=False,
                                   strides=strides, padding="SAME", name="conv2")
        self.bn2 = layers.BatchNormalization(momentum=0.9, epsilon=1e-5, name="conv2/BatchNorm")
        
        self.conv3 = layers.Conv2D(out_channel * self.expansion, kernel_size=1, use_bias=False, name="conv3")
        self.bn3 = layers.BatchNormalization(momentum=0.9, epsilon=1e-5, name="conv3/BatchNorm")
        
        self.relu = layers.ReLU()
        self.downsample = downsample
        self.add = layers.Add()
       
    
    def call(self, inputs, training = False):
        identity = inputs
        if self.downsample is not None:
            identity = self.downsample(inputs)
            
        x = self.conv1(inputs)
        x = self.bn1(x, training = training)
        x = self.relu(x)
        
        x = self.conv2(x)
        x = self.bn2(x, training = training)
        x = self.relu(x)
        
        x = self.conv3(x)
        x = self.bn3(x, training = training)
        
        x = self.add([x, identity])
        x = self.relu(x)
        
        return x
    
def _make_layer(block, in_channel, channel, block_num, name, strides=1):
    downsample = None
    if strides != 1 or in_channel != channel * block.expansion:
        downsample = Sequential([
            layers.Conv2D(channel * block.expansion, kernel_size=1, strides=strides,
                          use_bias=False, name="conv1"),
            layers.BatchNormalization(momentum=0.9, epsilon=1.001e-5, name= "BatchNorm")
        ], name="shortcut")
    
    layers_list = []
    layers_list.append(block(channel, downsample= downsample, strides=strides, name="unit_1"))
    
    for index in range(1, block_num):
        layers_list.append(block(channel, name="unit_" + str(index + 1)))
        
    
    return Sequential(layers_list, name=name)


def _resnet(block, blocks_num, im_width = 224, im_height = 224, num_classes=1000, include_top=True):
    input_image = layers.Input(shape=(im_height, im_width, 3), dtype="float32")
    x = layers.Conv2D(filters=64, kernel_size=7, strides=2,
                      padding="SAME", use_bias=False, name="conv1")(input_image)
    
    x = layers.BatchNormalization(momentum=0.9, epsilon=1e-5, name="conv1/BatchNorm")(x)
    x = layers.ReLU()(x)
    x = layers.MaxPooling2D(pool_size=3, strides=2, padding="SAME")(x)
    
    x = _make_layer(block, x.shape[-1], 64, block_num=[0], name="block1")(x)
    x = _make_layer(block, x.shape[-1], 128, block_num=[1], strides= 2, name="block2")(x)
    x = _make_layer(block, x.shape[-1], 256, block_num=[2], strides= 2, name="block3")(x)
    x = _make_layer(block, x.shape[-1], 512, block_num=[3], strides= 2, name="block4")(x)
    
    if include_top:
        x = layers.GlobalAvgPool2D()(x)
        x = layers.Dense(num_classes, name="logits")(x)
        predict = layers.Softmax()(x)
    else:
        predict = x

    
    model = Model(inputs=input_image, outputs = predict)
    
    return model

def resnet34(im_width=224, im_height=224, num_classes=1000, include_top=True):
    return _resnet(BasicBlock, [3, 4, 6, 3], im_width, im_height, num_classes, include_top)


def resnet50(im_width=224, im_height=224, num_classes=1000, include_top=True):
    return _resnet(Bottleneck, [3, 4, 6, 3], im_width, im_height, num_classes, include_top)


def resnet101(im_width=224, im_height=224, num_classes=1000, include_top=True):
    return _resnet(Bottleneck, [3, 4, 23, 3], im_width, im_height, num_classes, include_top)
        

In [ ]:
feature = resnet50(num_classes = 5, include_top = False)

In [ ]:
pre_weights_path = './pretrain_weights.ckpt'
feature.load_weights(pre_weights_path)
feature.trainable =False
feature.summary()

In [ ]:
model = tf.keras.Sequential([feature,
                             tf.keras.layers.GlobalAvgPool2D(),
                             tf.keras.layers.Dropout(rate=0.5),
                             tf.keras.layers.Dense(1024, activation="relu"),
                             tf.keras.layers.Dropout(rate=0.5),
                             tf.keras.layers.Dense(num_classes),
                             tf.keras.layers.Softmax()])
model.summary()

In [ ]:
loss_object = tf.keras.losses.CategoricalCrossentropy(from_logits=False)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0002)

train_loss = tf.keras.metrics.Mean(name="train_loss")
train_accuracy = tf.keras.metrics.CategoricalAccuracy(name='val_accuracy')

val_loss = tf.keras.metrics.Mean(name="val_loss")
val_accuracy = tf.keras.metrics.CategoricalAccuracy(name='val_accuracy')



In [ ]:
 @tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        output = model(images, training=True)
        loss = loss_object(labels, output)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, output)

@tf.function
def val_step(images, labels):
    output = model(images, training=False)
    loss = loss_object(labels, output)

    val_loss(loss)
    val_accuracy(labels, output)

In [ ]:
 best_val_acc = 0.
for epoch in range(epochs):
    train_loss.reset_states()  # clear history info
    train_accuracy.reset_states()  # clear history info
    val_loss.reset_states()  # clear history info
    val_accuracy.reset_states()  # clear history info

    # train
    train_bar = tqdm(train_dataset, file=sys.stdout)
    for images, labels in train_bar:
        train_step(images, labels)

        # print train process
        train_bar.desc = "train epoch[{}/{}] loss:{:.3f}, acc:{:.3f}".format(epoch + 1,
                                                                                 epochs,
                                                                                 train_loss.result(),
                                                                                 train_accuracy.result())

    optimizer.learning_rate = scheduler(epoch)

    # validate
    val_bar = tqdm(range(total_val // batch_size), file=sys.stdout)
    for step in val_bar:
        test_images, test_labels = next(val_data_gen)
        val_step(test_images, test_labels)

        # print val process
        val_bar.desc = "valid epoch[{}/{}] loss:{:.3f}, acc:{:.3f}".format(epoch + 1,
                                                                               epochs,
                                                                               val_loss.result(),
                                                                               val_accuracy.result())

    # only save best weights
    if val_accuracy.result() > best_val_acc:
        best_val_acc = val_accuracy.result()
        model.save_weights("./save_weights/resNet_50.ckpt", save_format="tf")


In [ ]:
image_path = '../input/flower/flower.jpg'
assert os.path.exists(image_path),"file: '{}' dose not exist.".format(image_path)
img = Image.open(image_path)
img = img.resize((im_width, im_height))
plt.imshow(img)

img = np.array(img).astype(np.float32)
img = img - [_R_MEAN, _G_MEAN, _B_MEAN]
img = (np.expand_dims(img, 0))

json_path = './class_indices.json'
assert os.path.exists(json_path),"file: '{}' does not exist.".format(json_path)

with open(json_path, "r") as f:
    class_indict = json.load(f)
    
feature = resnet50(num_classes = 5, include_top = False)
feature.trainable = False
# load weights
weights_path = './save_weights/resNet_50.ckpt'
assert len(glob.glob(weights_path+"*")), "cannot find {}".format(weights_path)
model.load_weights(weights_path)

# prediction
result = np.squeeze(model.predict(img))
predict_class = np.argmax(result)

print_res = "class: {}   prob: {:.3}".format(class_indict[str(predict_class)],
                                             result[predict_class])
plt.title(print_res)
for i in range(len(result)):
    print("class: {:10}   prob: {:.3}".format(class_indict[str(i)],
                                              result[i]))
plt.show()

